<a href="https://colab.research.google.com/github/shiftkey-labs/GenAI-Course/blob/main/gen_ai_book_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine Tuning

In [2]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0", split="train")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [12]:
def preprocess_function(examples):
    inputs = [doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [19]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
small_dataset = dataset.train_test_split(test_size=0.99)['train']  # This will keep only 1% of the dataset

In [21]:
tokenized_small_dataset = small_dataset.map(preprocess_function, batched=True)

In [22]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_small_dataset,
    tokenizer=tokenizer,
)

trainer.train()

Step,Training Loss
500,1.276000
1000,1.287600


TrainOutput(global_step=1077, training_loss=1.2797295403015647, metrics={'train_runtime': 447.6864, 'train_samples_per_second': 19.239, 'train_steps_per_second': 2.406, 'total_flos': 1601074917015552.0, 'train_loss': 1.2797295403015647, 'epoch': 3.0})

In [23]:
metrics = trainer.evaluate()
print(metrics)

ValueError: Trainer: evaluation requires an eval_dataset.

In [25]:
model.save_pretrained('./fine-tuned-flan-t5')
tokenizer.save_pretrained('./fine-tuned-flan-t5')

('./fine-tuned-flan-t5/tokenizer_config.json',
 './fine-tuned-flan-t5/special_tokens_map.json',
 './fine-tuned-flan-t5/spiece.model',
 './fine-tuned-flan-t5/added_tokens.json',
 './fine-tuned-flan-t5/tokenizer.json')

In [26]:
def summarize(text):
  inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
  summary_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
  return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

text = "The global pandemic has significantly impacted the way we work, leading to an unprecedented shift towards remote work. Companies around the world have had to adapt to new working conditions, often at a rapid pace. This transition has brought about both challenges and opportunities. While some employees enjoy the flexibility of working from home, others struggle with isolation and the blurring of work-life boundaries. Moreover, companies are now rethinking their long-term strategies, with many considering permanent remote work policies. However, this shift also raises concerns about maintaining company culture and ensuring effective collaboration among teams. As businesses navigate this new landscape, the ability to adapt and innovate will be key to their success."
print(summarize(text))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)